### Noise Gaussian

In [5]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx,load_pickle,is_low_noise_or,get_multiplier)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   


root_dir_output_low = f'/workspace/data/output/low_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_low}/NLM'
dir_out_geonlm =  f'{root_dir_output_low}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_low}/BM3D'
dir_out_results =  f'{root_dir_output_low}/results'


array_dir = read_directories(dir_images_general)

array_nln_low_filtereds = []

for file in array_dir:

    file_name = file
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)  # retorna float em [0,1]

    # >>> NORMALIZAÇÃO CRÍTICA: leve tudo para [0,255] float32 <<<
    if img.dtype.kind == 'f':                     # tipicamente após rgb2gray
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
    else:
        img = img.astype(np.float32)

    m, n = img.shape

    # Ruído (σ em 0–255, sem alterar seus valores)
    ruidosa = add_low_noise_gaussian(img)

    # Clipa para [0,255]
    ruidosa = np.clip(ruidosa, 0, 255)

    # CPU/GPU
    img_noisse_gaussian_np = ruidosa.astype(np.float32)
    estimated_sigma_gaussian_np = estimate_sigma(img_noisse_gaussian_np)  # data_range implícito 255
    img_gpu_noisse_gaussian = cp.array(img_noisse_gaussian_np)

    h_nlm = compute_adaptive_q(estimated_sigma_gaussian_np)
    q_nlm_candidates = np.array([h_nlm + delta for delta in range(-100, 25, 1)])

    # IMPORTANTE: passe a imagem de referência também em [0,255]
    img_filtered_nlm, nlm_h, psnr_nlm, ssim_nlm, score_nlm = select_best_h_using_adaptive_q(
        image=img,                          # agora está em [0,255] float32
        image_gpu=img_gpu_noisse_gaussian,  # idem
        q_nlm_candidates=q_nlm_candidates,
        f=4, t=7,
        alpha=0.5,
    )

    dct = {
        'img_noisse_gaussian_np': img_noisse_gaussian_np,
        'img_filtered_nlm': img_filtered_nlm,
        'estimated_sigma_gaussian': estimated_sigma_gaussian_np,
        'nlm_h': nlm_h,
        'psnr_nlm': psnr_nlm,
        'ssim_nlm': ssim_nlm,
        'score_nlm': score_nlm,
        'file_name': file_name,
    }
    array_nln_low_filtereds.append(dct)

    skimage.io.imsave(f'{dir_out_nlm}/{file_name}',np.clip(img_filtered_nlm, 0, 255).astype(np.uint8))
save_pickle(array_nln_low_filtereds, dir_out_results, "array_nln_low_filtereds.pkl")

h = 27.00 | PSNR = 34.27 | SSIM = 0.9361 | Score = 63.94
h = 28.00 | PSNR = 34.31 | SSIM = 0.9380 | Score = 64.05
h = 29.00 | PSNR = 34.35 | SSIM = 0.9400 | Score = 64.17
h = 30.00 | PSNR = 34.39 | SSIM = 0.9419 | Score = 64.29
h = 31.00 | PSNR = 34.43 | SSIM = 0.9436 | Score = 64.40
h = 32.00 | PSNR = 34.47 | SSIM = 0.9452 | Score = 64.49
h = 33.00 | PSNR = 34.50 | SSIM = 0.9466 | Score = 64.58
h = 34.00 | PSNR = 34.53 | SSIM = 0.9476 | Score = 64.65
h = 35.00 | PSNR = 34.56 | SSIM = 0.9485 | Score = 64.71
h = 36.00 | PSNR = 34.59 | SSIM = 0.9492 | Score = 64.75
h = 37.00 | PSNR = 34.61 | SSIM = 0.9497 | Score = 64.79
h = 38.00 | PSNR = 34.63 | SSIM = 0.9501 | Score = 64.82
h = 39.00 | PSNR = 34.66 | SSIM = 0.9504 | Score = 64.85
h = 40.00 | PSNR = 34.68 | SSIM = 0.9507 | Score = 64.87
h = 41.00 | PSNR = 34.71 | SSIM = 0.9509 | Score = 64.90
h = 42.00 | PSNR = 34.73 | SSIM = 0.9511 | Score = 64.92
h = 43.00 | PSNR = 34.76 | SSIM = 0.9512 | Score = 64.94
h = 44.00 | PSNR = 34.79 | SSIM

PosixPath('/workspace/data/output/low_noisy/results/array_nln_low_filtereds.pkl')

### GEONLM BM3D

In [6]:
from geonlm_functions import run_geonlm_pipeline
import time
from Utils import (save_results_to_xlsx, load_pickle)
vetor = load_pickle(dir_out_results, "array_nln_low_filtereds.pkl")
from bm3d import bm3d, BM3DProfile
from skimage.restoration import estimate_sigma
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.morphology import disk

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

array_gnlm_bm3d_low_filtereds = []
for array in vetor:

    img_noisse_gaussian_np =array['img_noisse_gaussian_np']
    
    nlm_h =array['nlm_h']
    file_name =array['file_name']
    psnr_nlm =array['psnr_nlm']
    ssim_nlm =array['ssim_nlm']
    score_nlm =array['score_nlm']
    estimated_sigma_gaussian = array['estimated_sigma_gaussian']

    print(f'filename: {file_name}')
    print(f'psnr_nlm: {psnr_nlm}')
    print(f'ssim_nlm: {ssim_nlm}')
    print(f'score_nlm: {score_nlm}')

    
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)  # retorna float em [0,1]

    # >>> NORMALIZAÇÃO CRÍTICA: leve tudo para [0,255] float32 <<<
    if img.dtype.kind == 'f':                     # tipicamente após rgb2gray
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
    else:
        img = img.astype(np.float32)
    
    # Parâmetros
    f = 4
    t = 7
    nn = 10  


    ini = time.time()    
    mult = get_multiplier(nlm_h, estimated_sigma_gaussian)
    print(f'multiplier: {mult}')
    print('h_nlm:', nlm_h)
    print('estimated_sigma_gaussian:', estimated_sigma_gaussian)
    img_filtered_gnlm, h_gnlm, psnr_gnlm, ssim_gnlm, score_gnlm = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)   
    skimage.io.imsave(f'{dir_out_geonlm}/{file_name}', img_filtered_gnlm.astype(np.uint8)) 
    end = time.time()
    time_geonlm = end - ini  

    img_bm3d = img.copy()

    if len(img_bm3d.shape) > 2:
        img_bm3d = skimage.color.rgb2gray(img_bm3d)
        img_bm3d = 255 * img_bm3d
    # Remove frames extras, se existirem (e.g. GIF com shape (1, H, W) ou (N, H, W, 3))
    if img_bm3d.ndim == 4:
        img_bm3d = img_bm3d[0]
    elif img_bm3d.ndim == 3 and img_bm3d.shape[2] != 3:
        img_bm3d = np.squeeze(img_bm3d)

    # Converte para escala de cinza, se necessário
    if img_bm3d.ndim == 3 and img_bm3d.shape[2] == 3:
        img_bm3d_gray = rgb2gray(img_bm3d)  # retorna float64 em [0, 1]
    else:
        img_bm3d_gray = img_bm3d.astype(np.float32) / 255.0  # já está em cinza
     # Agora está garantidamente em escala de cinza float [0,1]
    img_bm3d_gray = np.clip(img_bm3d_gray * 255, 0, 255).astype(np.uint8)
    
     # 1. Normaliza a imagem ruidosa para intervalo [0, 1]
    ruidosa_normalizada = img_noisse_gaussian_np.astype(np.float32) / 255.0

    sigma_est = estimate_sigma(ruidosa_normalizada, channel_axis=None)

    inicio = time.time()
    # 3. Define o perfil padrão do BM3D
    perfil_bm3d = BM3DProfile()
    fim = time.time() 
    time_bm3d = fim - inicio

    # 4. Aplica o BM3D com os argumentos definidos
    denoised = bm3d(
        ruidosa_normalizada,
        sigma_psd=sigma_est,
        profile=perfil_bm3d
    )
    denoised_sq = np.squeeze(denoised)   

    skimage.io.imsave(f'{dir_out_bm3d}/{file_name}',  np.clip(denoised_sq * 255, 0, 255).astype(np.uint8))

    
    psnr_bm3d = peak_signal_noise_ratio(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    ssim_bm3d = structural_similarity(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    score_bm3d = 0.5 * psnr_bm3d + 0.5 * (ssim_bm3d * 100)    
   

    dict = {     
        'nlm_h':nlm_h, 
        'h_gnlm':h_gnlm,       
        'estimated_sigma_gaussian':estimated_sigma_gaussian,      

        'ssim_nlm':ssim_nlm,
        'ssim_gnlm':ssim_gnlm,      
        'ssim_bm3d': ssim_bm3d,

        'psnr_nlm': psnr_nlm,
        'psnr_gnlm':psnr_gnlm,      
        'psnr_bm3d': psnr_bm3d,
        
        'score_nlm':score_nlm,   
        'score_gnlm':score_gnlm,      
        'score_bm3d': score_bm3d,   
       
        'file_name':file_name,

    }
    array_gnlm_bm3d_low_filtereds.append(dict)
save_pickle(array_gnlm_bm3d_low_filtereds, dir_out_results, "results_gaussian_gnlm_bm3d_low.pkl")
save_results_to_xlsx(array_gnlm_bm3d_low_filtereds, dir_out_results, 'gnlm_bm3d_low_filtereds.xlsx')

filename: 36.gif
psnr_nlm: 35.06239597904779
ssim_nlm: 0.9513546519592656
score_nlm: 65.09893058748717
multiplier: 1.4
h_nlm: 53
estimated_sigma_gaussian: 7.3315997195438625

Executando GEONLM com h = 74.20 (base 53 * 1.4)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 35.40, SSIM: 0.9537, Score: 65.38
filename: 39.gif
psnr_nlm: 37.5619957597427
ssim_nlm: 0.9445482577377583
score_nlm: 66.00841076675925
multiplier: 1.4
h_nlm: 47
estimated_sigma_gaussian: 5.69203227072998

Executando GEONLM com h = 65.80 (base 47 * 1.4)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 38.00, SSIM: 0.9454, Score: 66.27
filename: 4.gif
psnr_nlm: 35.73709883303429
ssim_nlm: 0.9288010443827835
score_nlm: 64.30860163565632
multiplier: 1.4
h_nlm: 48
estimated_sigma_gaussian: 5.939186487254527

Executando GEONLM com h = 67.20 (base 48 * 1.4)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 36.48, SSIM: 0.9372, Score: 65.10
filename: 3.gif
psnr_nlm: 36.55891293117647
ssim_nlm: 0.9405830727849273
score_nlm: 65.308610

PosixPath('/workspace/data/output/low_noisy/results/gnlm_bm3d_low_filtereds.xlsx')